In [ ]:
import pm4py 
bpmn_model = pm4py.read_bpmn('BPMN.bpmn')

net, im , fm = pm4py.convert_to_petri_net(bpmn_model)


In [ ]:
from core_simulator import SimulationEngine
engine = SimulationEngine(net, im, fm)
business_log = engine.run_simulation()

In [ ]:
print(business_log[0]) # datetime.datetime(1970, 4, 27, 0, 31, 12, tzinfo=datetime.timezone.utc)

In [ ]:
len(business_log)

In [ ]:
import pandas as pd
df = pd.DataFrame(business_log)

In [ ]:
df

### Analyze resource allocation 

- Resources were allocated based on resource availability probabilities
- Check against probabilies returned compliance of 92.17%

In [ ]:
from collections import Counter

# If business_log is a list of multiple trace dictionaries
all_resources = [
    event['org:resource'] 
    for trace in business_log 
    for event in trace['events']
]

total_counts = Counter(all_resources)
print(f"Total Resource Distribution: {total_counts}")

In [ ]:
def verify_stochastic_compliance(simulated_log, availability_probs):
    violations = 0
    total_human_events = 0

    for trace in simulated_log:
        for event in trace['events']:
            res = event['org:resource']
            ts = event['time:timestamp']
            
            # We only check human resources, as 'System' is our fallback
            if res != "System":
                total_human_events += 1
                # Check the (Resource, Weekday, Hour) triplet
                prob = availability_probs.get((res, ts.weekday(), ts.hour), 0)
                
                # If prob is 0, the resource worked when they never did in the log
                if prob == 0:
                    violations += 1

    print(f"--- Advanced Stochastic Report ---")
    print(f"Total Human Events: {total_human_events}")
    print(f"Historical Probability Violations: {violations}")
    if total_human_events > 0:
        print(f"Advanced Compliance: {((total_human_events - violations) / total_human_events) * 100:.2f}%")

In [ ]:
import json

def load_availability_from_json(filename="resource_availability.json"):
        with open(filename, 'r') as f:
            data = json.load(f)
        
        # Reconstruct the original tuple keys
        reconstructed = {}
        for k, v in data.items():
            parts = k.split('|')
            # parts[0] is resource, parts[1] is weekday (int), parts[2] is hour (int)
            reconstructed[(parts[0], int(parts[1]), int(parts[2]))] = v
        return reconstructed

In [ ]:
availability = load_availability_from_json()


In [ ]:
verify_stochastic_compliance(business_log, availability)